# Time Series Analysis
### **Authors**

Zoe Zhou 

### Objectives
In this assignment, you will perform time series plotting for a region of your choice.

### 1. Access data through Amazon Web Services cloud server
- Locate monthly data for a historical simulation run with any two CMIP6 models 
- Variables to analyze: surface air temperature (tas) and sea surface temperature (tos)

In [1]:
# Import packages
import xarray as xr
import matplotlib.pyplot as plt
import intake
import s3fs

- Search catalog for models for source_id, and ensemble member

In [4]:
# Search through catalog, find all historical simulations
# ("activity_id=CMIP", "experiment_id=historical")
res = catalog.search(activity_id="CMIP", experiment_id="historical")

# Convert to a data frame
res_df = res.df

# Get unique model names in the set of search results
models = res_df.source_id.unique()

# Print list of model names
print(models)

['GFDL-CM4' 'GFDL-ESM4' 'IPSL-CM6A-LR' 'GISS-E2-1-G' 'CNRM-CM6-1'
 'BCC-CSM2-MR' 'BCC-ESM1' 'CNRM-ESM2-1' 'MIROC6' 'AWI-CM-1-1-MR'
 'MRI-ESM2-0' 'CESM2-WACCM' 'CanESM5' 'CESM2' 'SAM0-UNICON' 'GISS-E2-1-H'
 'UKESM1-0-LL' 'EC-Earth3' 'CanESM5-CanOE' 'INM-CM4-8' 'EC-Earth3-Veg'
 'INM-CM5-0' 'HadGEM3-GC31-LL' 'MPI-ESM-1-2-HAM' 'NESM3' 'CAMS-CSM1-0'
 'MPI-ESM1-2-LR' 'MPI-ESM1-2-HR' 'E3SM-1-0' 'MCM-UA-1-0' 'NorESM2-LM'
 'GISS-E2-1-G-CC' 'FGOALS-g3' 'FGOALS-f3-L' 'MIROC-ES2L' 'KACE-1-0-G'
 'NorCPM1' 'CNRM-CM6-1-HR' 'KIOST-ESM' 'NorESM2-MM' 'ACCESS-CM2'
 'FIO-ESM-2-0' 'ACCESS-ESM1-5' 'CESM2-FV2' 'CESM2-WACCM-FV2' 'GISS-E2-2-H'
 'E3SM-1-1' 'HadGEM3-GC31-MM' 'IITM-ESM' 'CIESM' 'E3SM-1-1-ECA'
 'AWI-ESM-1-1-LR' 'EC-Earth3-Veg-LR' 'TaiESM1' 'CAS-ESM2-0' 'CMCC-CM2-SR5'
 'EC-Earth3-AerChem' 'IPSL-CM5A2-INCA' 'CMCC-CM2-HR4' 'EC-Earth3P-VHR'
 'EC-Earth3-CC' 'CMCC-ESM2' 'MIROC-ES2H' 'ICON-ESM-LR' 'IPSL-CM6A-LR-INCA']


- Choose CanESM5 and ACCESS-ESM1-5

In [7]:
# Search through catalog, find all historical simulations with CanESM5
# ("activity_id=CMIP", "experiment_id=historical", "source_id=CanESM5")
res = catalog.search(activity_id="CMIP", experiment_id="historical", source_id="CanESM5")

# Convert to a data frame
res_df = res.df

# Print all unique ensemble members ("member_id")
members = res_df.member_id.unique()

print(members)

['r24i1p1f1' 'r25i1p1f1' 'r14i1p1f1' 'r2i1p1f1' 'r17i1p1f1' 'r10i1p1f1'
 'r13i1p1f1' 'r7i1p1f1' 'r6i1p1f1' 'r5i1p1f1' 'r3i1p1f1' 'r22i1p1f1'
 'r23i1p1f1' 'r8i1p1f1' 'r11i1p1f1' 'r12i1p1f1' 'r15i1p1f1' 'r19i1p1f1'
 'r16i1p1f1' 'r1i1p1f1' 'r9i1p1f1' 'r18i1p1f1' 'r4i1p1f1' 'r21i1p1f1'
 'r20i1p1f1' 'r11i1p2f1' 'r10i1p2f1' 'r7i1p2f1' 'r9i1p2f1' 'r8i1p2f1'
 'r4i1p2f1' 'r40i1p2f1' 'r3i1p2f1' 'r6i1p2f1' 'r24i1p2f1' 'r13i1p2f1'
 'r12i1p2f1' 'r5i1p2f1' 'r31i1p2f1' 'r30i1p2f1' 'r32i1p2f1' 'r29i1p2f1'
 'r28i1p2f1' 'r2i1p2f1' 'r22i1p2f1' 'r23i1p2f1' 'r26i1p2f1' 'r27i1p2f1'
 'r25i1p2f1' 'r37i1p2f1' 'r38i1p2f1' 'r39i1p2f1' 'r35i1p2f1' 'r34i1p2f1'
 'r36i1p2f1' 'r33i1p2f1' 'r1i1p2f1' 'r18i1p2f1' 'r19i1p2f1' 'r14i1p2f1'
 'r15i1p2f1' 'r17i1p2f1' 'r16i1p2f1' 'r21i1p2f1' 'r20i1p2f1']


In [21]:
# Search through catalog, find all historical simulations with ACCESS-ESM1-5
res_ACCESS = catalog.search(activity_id="CMIP", experiment_id="historical", source_id="CNRM-CM6-1")

# Convert to a data frame
res_ACCESS_df = res.df

# Print all unique ensemble members ("member_id")
members_ACCESS = res_ACCESS_df.member_id.unique()

print(members_ACCESS)

['r10i1p1f1']


- Both models have historical simulations with member_id = r10i1p1f1

## CanESM5 model data

In [11]:
# Specify search terms to query catalog for CanESM5 data
# activity_id: which project do you want? CMIP = historical data, ScenarioMIP = future projections
activity_ids = ['ScenarioMIP', 'CMIP'] 

# source_id: which model do you want? 
source_id = ['CanESM5']

# experiment_id: what experimental configuration do you want? Here we want historical and the four main SSPs
experiment_ids = ['historical', 'ssp126', 'ssp370']

# member_id: which ensemble member do you want? Here we want r10i1p1f1
member_id = 'r10i1p1f1'

# table_id: which part of the Earth system and time resolution do you want? Here we want monthly atmosphere data
table_id = 'Amon' 

# variable_id: which climate variable do you want? Here we want surface air temperature
variable_id = ['tas', 'pr'] 


# Open the CMIP6 data catalog, store as a variable
catalog = intake.open_esm_datastore('https://cmip6-pds.s3.amazonaws.com/pangeo-cmip6.json')

# Search through catalog, store results in "res" variable
CanESM5_res = catalog.search(activity_id=activity_ids, source_id=source_id, experiment_id=experiment_ids, 
                     member_id=member_id, table_id=table_id, variable_id=variable_id)

# Display data frame associated with results
display(CanESM5_res.df)

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,zstore,dcpp_init_year,version
0,CMIP,CCCma,CanESM5,historical,r10i1p1f1,Amon,pr,gn,s3://cmip6-pds/CMIP6/CMIP/CCCma/CanESM5/histor...,NaN,20190429
1,CMIP,CCCma,CanESM5,historical,r10i1p1f1,Amon,tas,gn,s3://cmip6-pds/CMIP6/CMIP/CCCma/CanESM5/histor...,NaN,20190429
2,ScenarioMIP,CCCma,CanESM5,ssp370,r10i1p1f1,Amon,tas,gn,s3://cmip6-pds/CMIP6/ScenarioMIP/CCCma/CanESM5...,NaN,20190429
3,ScenarioMIP,CCCma,CanESM5,ssp370,r10i1p1f1,Amon,pr,gn,s3://cmip6-pds/CMIP6/ScenarioMIP/CCCma/CanESM5...,NaN,20190429
4,ScenarioMIP,CCCma,CanESM5,ssp126,r10i1p1f1,Amon,pr,gn,s3://cmip6-pds/CMIP6/ScenarioMIP/CCCma/CanESM5...,NaN,20190429
5,ScenarioMIP,CCCma,CanESM5,ssp126,r10i1p1f1,Amon,tas,gn,s3://cmip6-pds/CMIP6/ScenarioMIP/CCCma/CanESM5...,NaN,20190429


2. Read in data and store as an xarray object
Load in data from the historical simulation, as well as corresponding future projection information for two different future emissions scenarios, for each model.

In [12]:
# Read in just the historical data file
can_pr_hist_data = xr.open_zarr(CanESM5_res.df['zstore'][0], storage_options={'anon': True})
# Get data for SSP126
can_pr_ssp126_data = xr.open_zarr(CanESM5_res.df['zstore'][4], storage_options={'anon': True})
# Get data for SSP370
can_pr_ssp370_data = xr.open_zarr(CanESM5_res.df['zstore'][3], storage_options={'anon': True})

# Read in just the historical data file
can_tas_hist_data = xr.open_zarr(CanESM5_res.df['zstore'][1], storage_options={'anon': True})
# Get data for SSP126
can_tas_ssp126_data = xr.open_zarr(CanESM5_res.df['zstore'][5], storage_options={'anon': True})
# Get data for SSP370
can_tas_ssp370_data = xr.open_zarr(CanESM5_res.df['zstore'][2], storage_options={'anon': True})

## ACCESS-ESM1-5 Model Data

In [22]:
# Specify search terms to query catalog for CanESM5 data
# activity_id: which project do you want? CMIP = historical data, ScenarioMIP = future projections
activity_ids = ['ScenarioMIP', 'CMIP'] 

# source_id: which model do you want? 
source_id = ['CNRM-CM6-1']

# experiment_id: what experimental configuration do you want? Here we want historical and the four main SSPs
experiment_ids = ['historical', 'ssp126', 'ssp370']

# member_id: which ensemble member do you want? Here we want r10i1p1f1
member_id = 'r10i1p1f1'

# table_id: which part of the Earth system and time resolution do you want? Here we want monthly atmosphere data
table_id = 'Amon' 

# variable_id: which climate variable do you want? Here we want surface air temperature
variable_id = ['tas', 'pr'] 


# Open the CMIP6 data catalog, store as a variable
catalog = intake.open_esm_datastore('https://cmip6-pds.s3.amazonaws.com/pangeo-cmip6.json')

# Search through catalog, store results in "res" variable
ACCESS_res = catalog.search(activity_id=activity_ids, source_id=source_id, experiment_id=experiment_ids, 
                     member_id=member_id, table_id=table_id, variable_id=variable_id)

# Display data frame associated with results
display(ACCESS_res.df)

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,zstore,dcpp_init_year,version


2. Read in data and store as an xarray object
Load in data from the historical simulation, as well as corresponding future projection information for two different future emissions scenarios, for each model.


In [ ]:
# Read in just the historical data file
can_pr_hist_data = xr.open_zarr(CanESM5_res.df['zstore'][0], storage_options={'anon': True})
# Get data for SSP126
can_pr_ssp126_data = xr.open_zarr(CanESM5_res.df['zstore'][4], storage_options={'anon': True})
# Get data for SSP370
can_pr_ssp370_data = xr.open_zarr(CanESM5_res.df['zstore'][3], storage_options={'anon': True})

# Read in just the historical data file
can_tas_hist_data = xr.open_zarr(CanESM5_res.df['zstore'][1], storage_options={'anon': True})
# Get data for SSP126
can_tas_ssp126_data = xr.open_zarr(CanESM5_res.df['zstore'][5], storage_options={'anon': True})
# Get data for SSP370
can_tas_ssp370_data = xr.open_zarr(CanESM5_res.df['zstore'][2], storage_options={'anon': True})